In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Set paths to your dataset folders
train_dir = '/home/esidserver/datasets/testing_cls/train/'
val_dir = '/home/esidserver/datasets/testing_cls/val/'
test_dir = '/home/esidserver/datasets/testing_cls/test/'

# Image data generator with data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator for validation and test
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Data loaders
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Load the VGG19 model with pre-trained weights
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('vgg19_best_model.keras', save_best_only=True, monitor='val_loss')

# Train the model
history = model.fit(
    train_generator,
    epochs=250,
    validation_data=val_generator,
    callbacks=[early_stopping, model_checkpoint]
)


2024-09-19 11:33:11.173396: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-19 11:33:11.197754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 11:33:11.230213: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 11:33:11.239034: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-19 11:33:11.261383: I tensorflow/core/platform/cpu_feature_guar

Found 5193 images belonging to 4 classes.
Found 1484 images belonging to 4 classes.
Found 740 images belonging to 4 classes.


2024-09-19 11:33:13.653619: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22453 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:af:00.0, compute capability: 8.6


Epoch 1/250


/home/esidserver/miniforge3/envs/tf/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1726720400.945441 3678844 service.cc:146] XLA service 0x766f1c004e60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726720400.945500 3678844 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-09-19 11:33:21.031957: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-19 11:33:21.402854: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902
2

...

 17/163 ━━━━━━━━━━━━━━━━━━━━ 47s 327ms/step - accuracy: 0.2103 - loss: 1.4494

2024-09-19 11:33:33.617828: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_944', 12 bytes spill stores, 12 bytes spill loads



162/163 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step - accuracy: 0.3909 - loss: 1.3202

2024-09-19 11:34:31.009978: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_252', 12 bytes spill stores, 12 bytes spill loads



163/163 ━━━━━━━━━━━━━━━━━━━━ 78s 416ms/step - accuracy: 0.3922 - loss: 1.3191 - val_accuracy: 0.6550 - val_loss: 1.0660
Epoch 2/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 61s 360ms/step - accuracy: 0.6162 - loss: 1.0644 - val_accuracy: 0.7177 - val_loss: 0.8977
Epoch 3/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 60s 355ms/step - accuracy: 0.6661 - loss: 0.9382 - val_accuracy: 0.7271 - val_loss: 0.7956
Epoch 4/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 61s 356ms/step - accuracy: 0.6910 - loss: 0.8459 - val_accuracy: 0.7540 - val_loss: 0.7144
Epoch 5/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 62s 367ms/step - accuracy: 0.7083 - loss: 0.7927 - val_accuracy: 0.7823 - val_loss: 0.6455
Epoch 6/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 60s 354ms/step - accuracy: 0.7262 - loss: 0.7470 - val_accuracy: 0.7823 - val_loss: 0.6265
Epoch 7/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 60s 354ms/step - accuracy: 0.7175 - loss: 0.7198 - val_accuracy: 0.7931 - val_loss: 0.5815
Epoch 8/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 61s 359ms/step - accuracy: 0.7299 - loss: 0.702

In [2]:
# Fine-tuning: Unfreeze some layers of the base model
for layer in base_model.layers:
    layer.trainable = True

# Compile the model again
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training with fine-tuning for 100 epochs
history_finetune = model.fit(
    train_generator,
    epochs=100,  # Fine-tuning for 100 epochs
    validation_data=val_generator,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/100


2024-09-19 15:43:00.786545: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1654', 76 bytes spill stores, 76 bytes spill loads



163/163 ━━━━━━━━━━━━━━━━━━━━ 94s 421ms/step - accuracy: 0.8838 - loss: 0.3063 - val_accuracy: 0.9340 - val_loss: 0.1700
Epoch 2/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 60s 350ms/step - accuracy: 0.9391 - loss: 0.1726 - val_accuracy: 0.9609 - val_loss: 0.1001
Epoch 3/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 59s 349ms/step - accuracy: 0.9587 - loss: 0.1150 - val_accuracy: 0.9677 - val_loss: 0.0837
Epoch 4/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 60s 350ms/step - accuracy: 0.9618 - loss: 0.1045 - val_accuracy: 0.9764 - val_loss: 0.0617
Epoch 5/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 60s 352ms/step - accuracy: 0.9685 - loss: 0.0781 - val_accuracy: 0.9818 - val_loss: 0.0452
Epoch 6/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 59s 347ms/step - accuracy: 0.9715 - loss: 0.0739 - val_accuracy: 0.9825 - val_loss: 0.0516
Epoch 7/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 60s 350ms/step - accuracy: 0.9820 - loss: 0.0497 - val_accuracy: 0.9818 - val_loss: 0.0464
Epoch 8/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 60s 350ms/step - accuracy: 0.9844 - loss: 0.045

In [3]:
# Evaluate on the test set
test_loss, test_acc = model.evaluate(test_generator)

# Calculate final training and validation accuracy
train_acc = history_finetune.history['accuracy'][-1] if 'accuracy' in history_finetune.history else None
val_acc = history_finetune.history['val_accuracy'][-1] if 'val_accuracy' in history_finetune.history else None

# Print the results
print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Validation Accuracy: {val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

# Save the final model
model.save('vgg19_final_model.keras')

23/24 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.9957 - loss: 0.0209

2024-09-19 17:24:07.577769: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_252', 4 bytes spill stores, 4 bytes spill loads



24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 211ms/step - accuracy: 0.9957 - loss: 0.0208
Final Training Accuracy: 0.9952
Final Validation Accuracy: 0.9939
Test Accuracy: 0.9959, Test Loss: 0.0196


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

# Set paths to your dataset folders
train_dir = '/home/esidserver/datasets/testing_cls/training-v2/train/'
val_dir = '/home/esidserver/datasets/testing_cls/training-v2/val/'
test_dir = '/home/esidserver/datasets/testing_cls/training-v2/test/'

# Image data generator with data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator for validation and test
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Data loaders
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Important for confusion matrix to align predictions with true labels
)

# Load the VGG19 model with pre-trained weights
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('vgg19_best_model.keras', save_best_only=True, monitor='val_loss')

# Train the model
history = model.fit(
    train_generator,
    epochs=250,
    validation_data=val_generator,
    callbacks=[early_stopping, model_checkpoint]
)

Found 5193 images belonging to 4 classes.
Found 1484 images belonging to 4 classes.
Found 740 images belonging to 4 classes.
Epoch 1/250


/home/esidserver/miniforge3/envs/tf/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 66s 372ms/step - accuracy: 0.4363 - loss: 1.2814 - val_accuracy: 0.6530 - val_loss: 1.0325
Epoch 2/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 60s 349ms/step - accuracy: 0.6247 - loss: 1.0482 - val_accuracy: 0.7049 - val_loss: 0.8784
Epoch 3/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 60s 353ms/step - accuracy: 0.6708 - loss: 0.9308 - val_accuracy: 0.7278 - val_loss: 0.7738
Epoch 4/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 58s 343ms/step - accuracy: 0.6980 - loss: 0.8180 - val_accuracy: 0.7446 - val_loss: 0.7124
Epoch 5/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 59s 345ms/step - accuracy: 0.7237 - loss: 0.7651 - val_accuracy: 0.7796 - val_loss: 0.6381
Epoch 6/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 58s 342ms/step - accuracy: 0.7461 - loss: 0.7092 - val_accuracy: 0.7911 - val_loss: 0.5891
Epoch 7/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 58s 342ms/step - accuracy: 0.7464 - loss: 0.6934 - val_accuracy: 0.7999 - val_loss: 0.5528
Epoch 8/250
163/163 ━━━━━━━━━━━━━━━━━━━━ 58s 342ms/step - accuracy: 0.7603 - loss: 0.644

KeyboardInterrupt: 

: 

In [ ]:
# Fine-tuning: Unfreeze some layers of the base model
for layer in base_model.layers:
    layer.trainable = True

# Compile the model again
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training with fine-tuning for 100 epochs
history_finetune = model.fit(
    train_generator,
    epochs=100,  # Fine-tuning for 100 epochs
    validation_data=val_generator,
    callbacks=[early_stopping, model_checkpoint]
)

In [ ]:
# Evaluate on the test set
test_loss, test_acc = model.evaluate(test_generator)

# Calculate final training and validation accuracy
train_acc = history_finetune.history['accuracy'][-1] if 'accuracy' in history_finetune.history else None
val_acc = history_finetune.history['val_accuracy'][-1] if 'val_accuracy' in history_finetune.history else None

# Print the results
print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Validation Accuracy: {val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

# Save the final model
model.save('vgg19_final_model.keras')

In [ ]:

# Generate predictions for the test set
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get true labels
y_true = test_generator.classes

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
cm_labels = list(test_generator.class_indices.keys())

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=cm_labels)
disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
plt.title('Confusion Matrix')
plt.show()